In [1]:
import ee
import geemap

In [2]:
geemap.ee_initialize()

In [3]:
Map = geemap.Map(center=[1.3521, 103.8198], zoom=12)
Map

Map(center=[1.3521, 103.8198], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDa…

In [4]:
country_name = "Singapore"
country = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017').filter(
    ee.Filter.eq('country_na', country_name)
)
style = {'color': 'black', 'fillColor': '00000000'}
Map.addLayer(country.style(**style), {}, country_name)
Map.centerObject(country)
Map

Map(bottom=520649.0, center=[1.3521, 103.8198], controls=(WidgetControl(options=['position', 'transparent_bg']…

In [5]:
landsat_col_1985_1990 = (
    ee.ImageCollection("LANDSAT/LT05/C02/T1")
    .filterDate('1985-01-01', '1990-05-30')
    .filterBounds(country)
)

landsat_1985_1990 = ee.Algorithms.Landsat.simpleComposite(landsat_col_1985_1990).clipToCollection(country)

Map.addLayer(landsat_1985_1990, {'bands': ['B3', 'B2', 'B1'], 'min': 0, 'max': 80}, 'Landsat 1985-1990')

In [6]:
landsat_col_2022_2024 = (
    ee.ImageCollection("LANDSAT/LC09/C02/T1")
    .filterDate('2022-01-01', '2024-05-30')
    .filterBounds(country)
)

landsat_2022_2024 = ee.Algorithms.Landsat.simpleComposite(landsat_col_2022_2024).clipToCollection(country)

Map.addLayer(landsat_2022_2024, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 80}, 'Landsat 2022-2024')

In [7]:
Map.centerObject(country)
Map

Map(bottom=260488.0, center=[1.3426226258546694, 103.84561659965722], controls=(WidgetControl(options=['positi…

In [8]:
Map = geemap.Map(center=[1.3521, 103.8198], zoom=12)

left_layer = geemap.ee_tile_layer(
    landsat_1985_1990, {'bands': ['B3', 'B2', 'B1'], 'min': 0, 'max': 80}, 'landsat_1985_1990'
   )
right_layer = geemap.ee_tile_layer(
    landsat_2022_2024, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 80}, 'landsat_2022_2024'
)

Map.split_map(
    left_layer, right_layer, left_label='landsat_1985_1990', right_label='landsat_2022_2024'
)
Map.addLayer(country.style(**style), {}, country_name)
Map

Map(center=[1.3521, 103.8198], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zo…

In [9]:
ndvi_past = landsat_1985_1990.normalizedDifference(['B4', 'B3']).rename('ndvi')
ndvi_present = landsat_2022_2024.normalizedDifference(['B5', 'B4']).rename('ndvi')

In [10]:
Map = geemap.Map(center=[1.3521, 103.8198], zoom=12)
ndvi_vis = {'min': -1, 'max': 1, 'palette': ['#FE9929', '#FEE391', '#C7E9C0', '#1C9099', '#008837', '#00441B']}

left_layer = geemap.ee_tile_layer(ndvi_past, ndvi_vis, 'ndvi past')
right_layer = geemap.ee_tile_layer(ndvi_present, ndvi_vis, 'ndvi present')

Map.split_map(left_layer, right_layer, left_label='ndvi past', right_label='ndvi present')
Map.addLayer(country.style(**style), {}, country_name)
Map

Map(center=[1.3521, 103.8198], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zo…

In [11]:
threshold = 0.2
vegetation_past = ndvi_past.gt(threshold).selfMask()
vegetation_present = ndvi_present.gt(threshold).selfMask()

Map = geemap.Map(center=[1.3521, 103.8198], zoom=12)

left_layer = geemap.ee_tile_layer(vegetation_past, {'palette': 'blue'}, 'vegetation past')
right_layer = geemap.ee_tile_layer(vegetation_present, {'palette': 'red'}, 'vegetation present')
Map.split_map(left_layer, right_layer, left_label='vegetation past', right_label='vegetation present')

Map.addLayer(country.style(**style), {}, country_name)

Map

Map(center=[1.3521, 103.8198], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zo…

In [12]:
increased_vegetation = vegetation_present.unmask().subtract(vegetation_past.unmask()).gt(0.2).selfMask()
decreased_vegetation = vegetation_past.unmask().subtract(vegetation_present.unmask()).gt(0.2).selfMask()

In [13]:
Map = geemap.Map(center=[1.3521, 103.8198], zoom=12)

Map.addLayer(landsat_1985_1990, {'bands': ['B3', 'B2', 'B1'], 'min': 0, 'max': 80}, 'landsat_1985_1990', False)
Map.addLayer(landsat_2022_2024, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 80}, 'landsat_2022_2024', False)

Map.addLayer(increased_vegetation, {'palette': 'green'}, 'increased vegetation')
Map.addLayer(decreased_vegetation, {'palette': 'purple'}, 'decreased vegetation')
Map.addLayer(country.style(**style), {}, country_name)
Map

Map(center=[1.3521, 103.8198], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDa…

In [14]:
decreased_area = geemap.zonal_stats(
    decreased_vegetation, country, scale=1000, statistics_type='SUM', return_fc=True
)
geemap.ee_to_df(decreased_area)

Computing statistics ...


,abbreviati,country_co,country_na,sum,wld_rgn
0,Sing.,SN,Singapore,61.254902,SE Asia


In [15]:
increased_area = geemap.zonal_stats(
    increased_vegetation, country, scale=1000, statistics_type='SUM', return_fc=True
)
geemap.ee_to_df(increased_area)

Computing statistics ...


,abbreviati,country_co,country_na,sum,wld_rgn
0,Sing.,SN,Singapore,99.6,SE Asia


TIMELAPSE

In [16]:
Map = geemap.Map(center=[1.3521, 103.8198], zoom=12)
Map.addLayer(country.style(**style), {}, country_name)
Map.centerObject(country)
Map

Map(center=[1.3521, 103.8198], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDa…

In [17]:
roi = country.geometry()

timelapse = geemap.landsat_timelapse(
    roi=roi,
    out_gif='singapore_landsat_timelapse_19852024.gif',
    start_year=1985,
    end_year=2024,
    start_date='01-01',
    end_date='12-31',
    bands=['NIR', 'Red', 'Green'],
    frames_per_second=5,
    title='Singapore Landsat Timelapse',
    progress_bar_color='blue',
    mp4=False,
    fading=False,
)

geemap.show_image(timelapse)

Generating URL...
Please wait ...
The GIF image has been saved to: c:\Users\김태호\Desktop\24-1\위성빅데이터\위성과제 자료\singapore_landsat_timelapse_19852024.gif


Output()